# 인근 버스 역 정보 feature 추가

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from geopy.distance import geodesic

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# Data Load

In [2]:
df = pd.read_csv('../data/preprocessed/2_subway_preprocessed.csv')

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_60948/4275245894.py:1: DtypeWarning: Columns (17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/preprocessed/2_subway_preprocessed.csv')


In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,시군구 번지,"좌표X,좌표Y",가장 가까운 거리,index,역사_ID,역사명,호선,위도,경도,인근 지하철 역 개수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,서울특별시 강남구 개포동 658-1,"127.05721,37.476763",1127.738351,619.0,1026.0,구룡,분당선,37.486839,127.058856,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,서울특별시 중랑구 신내동 816,"127.1067198,37.6188699",732.482898,562.0,1311.0,신내,경춘선,37.612887,127.103218,4.0


In [5]:
bus = pd.read_csv('../data/bus_feature.csv')

In [9]:
bus.head()

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,중앙차로
2,100000003,1003,명륜3가.성대입구,126.998251,37.582581,중앙차로
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,중앙차로
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,중앙차로


# geopy를 사용하여 역과 아파트 사이의 거리 계산
- '좌표X', '좌표Y'를 합쳐서 '좌표X,좌표Y'변수를 생성한다.
    - 모든 데이터를 다 구하면 너무 오래 걸림
    - unique한 값을 구할 수 있다.
    - 연산 횟수를 줄일 수 있다.

- 각각의 좌표에 대해서 역과의 거리를 구한다.
- 다양한 feature를 생성한다.

In [6]:
# '좌표X', '좌표Y'를 합쳐서 하나의 변수 '좌표X,좌표Y'생성

df['좌표X,좌표Y'] = df['좌표X'].astype(str)+','+df['좌표Y'].astype(str)
print(len(df['좌표X,좌표Y'].unique()))

loc_df = pd.DataFrame(df['좌표X,좌표Y'].unique(), columns=['좌표X,좌표Y']) # '좌표X,좌표Y' 부분만 추출
loc_df

8921


,"좌표X,좌표Y"
0,"127.05721,37.476763"
1,"127.0559898,37.4838941"
2,"127.0766243,37.4962963"
3,"127.0585214,37.4800022"
4,"127.0680279,37.4878016"
...,...
8916,"126.960783522562,37.5003136768081"
8917,"126.935945913625,37.5448750548629"
8918,"127.063427393682,37.5487159907398"
8919,"126.9966,37.5645412"


In [7]:
# 좌표 X, Y 다시 분리

loc_df[['좌표X', '좌표Y']] = loc_df['좌표X,좌표Y'].apply(lambda x: pd.Series(x.split(',')))

loc_df['좌표X'] = loc_df['좌표X'].astype(float)
loc_df['좌표Y'] = loc_df['좌표Y'].astype(float)

loc_df

,"좌표X,좌표Y",좌표X,좌표Y
0,"127.05721,37.476763",127.057210,37.476763
1,"127.0559898,37.4838941",127.055990,37.483894
2,"127.0766243,37.4962963",127.076624,37.496296
3,"127.0585214,37.4800022",127.058521,37.480002
4,"127.0680279,37.4878016",127.068028,37.487802
...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314
8917,"126.935945913625,37.5448750548629",126.935946,37.544875
8918,"127.063427393682,37.5487159907398",127.063427,37.548716
8919,"126.9966,37.5645412",126.996600,37.564541


In [12]:
# 아파트와 인근 지하철 거리 정보 계산
# 반경 m이하의 정류장 개수
# 가장 가까운 정류장
# cols : 노드 ID	정류소번호	정류소명	X좌표	Y좌표	정류소 타입
m = 300

apart_loc = (loc_df.iloc[0]['좌표Y'], loc_df.iloc[0]['좌표X']) # 아파트 위치 정보
re = bus.apply(lambda loc: geodesic(apart_loc, (loc['Y좌표'], loc['X좌표'])).meters, axis=1) # 아파트와 모든 지하철 정류장 사이의 거리를 구한다.

# 의미있는 정보 표시
print(f'1.5km 이하에 위치한 역 개수: {(re < m).sum()}')
shortest = bus.iloc[re.idxmin()]
print(f'가장 가까운 역과의 거리: {re.min()/1000:.2f}km')
print('가장 가까운 정류장 이름:', shortest['정류소명'])
print('가장 가까운 정류장 타입:', shortest['정류소 타입'])

1.5km 이하에 위치한 역 개수: 7
가장 가까운 역과의 거리: 0.06km
가장 가까운 정류장 이름: 개포우성아파트
가장 가까운 정류장 타입: 일반차로


## 버스 정류장

In [14]:
def calculate_dist(loc_df, bus_df):
    apart_loc = (loc_df['좌표Y'], loc_df['좌표X'])
    distances = bus_df.apply(lambda loc: geodesic(apart_loc, (loc['Y좌표'], loc['X좌표'])).meters, axis=1)
    return distances

distances = loc_df.apply(calculate_dist, bus_df = bus, axis=1)

In [16]:
distances.shape

(8921, 12584)

In [17]:
distances.to_csv('./버스 정류장.csv', index=False)

In [18]:
loc_df['가장 가까운 거리'] = distances.min(axis=1)
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리
0,"127.05721,37.476763",127.057210,37.476763,61.895840
1,"127.0559898,37.4838941",127.055990,37.483894,98.497224
2,"127.0766243,37.4962963",127.076624,37.496296,202.274566
3,"127.0585214,37.4800022",127.058521,37.480002,234.803539
4,"127.0680279,37.4878016",127.068028,37.487802,131.024097
...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,10.216654
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,37.734667
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,96.262129
8919,"126.9966,37.5645412",126.996600,37.564541,138.510459


In [19]:
min_idx = distances.idxmin(axis=1)
bus.iloc[min_idx].reset_index()

,index,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,11345,122000333,23445,개포우성아파트,127.057864,37.476564,일반차로
1,11398,122000392,23515,우성3차아파트,127.057080,37.483713,일반차로
2,11403,122000397,23520,코원에너지서비스,127.076097,37.498070,일반차로
3,11356,122000345,23459,개포주공1단지아파트,127.056800,37.478392,일반차로
4,11255,122000249,23353,개포주공5단지경기여고,127.067471,37.486708,일반차로
...,...,...,...,...,...,...,...
8916,9431,119900080,20505,달마사,126.960695,37.500254,마을버스
8917,6426,113900145,14873,한국출판콘텐츠센터,126.935550,37.544747,마을버스
8918,1214,103000114,4213,성동세무서,127.064337,37.548239,일반차로
8919,537,101000065,2164,중구청앞.덕수중학교,126.998140,37.564775,일반차로


In [20]:
loc_df = pd.concat([loc_df, bus.iloc[min_idx].reset_index()], axis=1)
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리,index,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,"127.05721,37.476763",127.057210,37.476763,61.895840,11345,122000333,23445,개포우성아파트,127.057864,37.476564,일반차로
1,"127.0559898,37.4838941",127.055990,37.483894,98.497224,11398,122000392,23515,우성3차아파트,127.057080,37.483713,일반차로
2,"127.0766243,37.4962963",127.076624,37.496296,202.274566,11403,122000397,23520,코원에너지서비스,127.076097,37.498070,일반차로
3,"127.0585214,37.4800022",127.058521,37.480002,234.803539,11356,122000345,23459,개포주공1단지아파트,127.056800,37.478392,일반차로
4,"127.0680279,37.4878016",127.068028,37.487802,131.024097,11255,122000249,23353,개포주공5단지경기여고,127.067471,37.486708,일반차로
...,...,...,...,...,...,...,...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,10.216654,9431,119900080,20505,달마사,126.960695,37.500254,마을버스
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,37.734667,6426,113900145,14873,한국출판콘텐츠센터,126.935550,37.544747,마을버스
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,96.262129,1214,103000114,4213,성동세무서,127.064337,37.548239,일반차로
8919,"126.9966,37.5645412",126.996600,37.564541,138.510459,537,101000065,2164,중구청앞.덕수중학교,126.998140,37.564775,일반차로


In [22]:
(distances<=300).sum(axis=1)

0        7
1        9
2        2
3        2
4       11
        ..
8916    13
8917     9
8918     9
8919     4
8920     9
Length: 8921, dtype: int64

In [23]:
loc_df['인근 버스 정류장 개수'] = (distances<=300).sum(axis=1)

In [24]:
loc_df

,"좌표X,좌표Y",좌표X,좌표Y,가장 가까운 거리,index,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입,인근 버스 정류장 개수
0,"127.05721,37.476763",127.057210,37.476763,61.895840,11345,122000333,23445,개포우성아파트,127.057864,37.476564,일반차로,7
1,"127.0559898,37.4838941",127.055990,37.483894,98.497224,11398,122000392,23515,우성3차아파트,127.057080,37.483713,일반차로,9
2,"127.0766243,37.4962963",127.076624,37.496296,202.274566,11403,122000397,23520,코원에너지서비스,127.076097,37.498070,일반차로,2
3,"127.0585214,37.4800022",127.058521,37.480002,234.803539,11356,122000345,23459,개포주공1단지아파트,127.056800,37.478392,일반차로,2
4,"127.0680279,37.4878016",127.068028,37.487802,131.024097,11255,122000249,23353,개포주공5단지경기여고,127.067471,37.486708,일반차로,11
...,...,...,...,...,...,...,...,...,...,...,...,...
8916,"126.960783522562,37.5003136768081",126.960784,37.500314,10.216654,9431,119900080,20505,달마사,126.960695,37.500254,마을버스,13
8917,"126.935945913625,37.5448750548629",126.935946,37.544875,37.734667,6426,113900145,14873,한국출판콘텐츠센터,126.935550,37.544747,마을버스,9
8918,"127.063427393682,37.5487159907398",127.063427,37.548716,96.262129,1214,103000114,4213,성동세무서,127.064337,37.548239,일반차로,9
8919,"126.9966,37.5645412",126.996600,37.564541,138.510459,537,101000065,2164,중구청앞.덕수중학교,126.998140,37.564775,일반차로,4


### 원본 데이터에 적용

In [25]:
# '시군구 번지'에 맞춰 X, Y 를 매핑

shortest_dist_mapping = pd.Series({locate: shortest_dist for locate, shortest_dist in zip(loc_df['좌표X,좌표Y'], loc_df['가장 가까운 거리'])})
idx_mapping = pd.Series({locate: idx for locate, idx in zip(loc_df['좌표X,좌표Y'], loc_df['index'])})
ID_mapping = pd.Series({locate: ID for locate, ID in zip(loc_df['좌표X,좌표Y'], loc_df['노드 ID'])})
station_num_mapping = pd.Series({locate: station_num for locate, station_num in zip(loc_df['좌표X,좌표Y'], loc_df['정류소번호'])})
station_name_mapping = pd.Series({locate: station_name for locate, station_name in zip(loc_df['좌표X,좌표Y'], loc_df['정류소명'])})
station_type_mapping = pd.Series({locate: station_type for locate, station_type in zip(loc_df['좌표X,좌표Y'], loc_df['정류소 타입'])})
X_mapping = pd.Series({locate: latitude for locate, latitude in zip(loc_df['좌표X,좌표Y'], loc_df['X좌표'])})
Y_mapping = pd.Series({locate: longitude for locate, longitude in zip(loc_df['좌표X,좌표Y'], loc_df['Y좌표'])})
nums_mapping = pd.Series({locate: nums for locate, nums in zip(loc_df['좌표X,좌표Y'], loc_df['인근 버스 정류장 개수'])})

In [26]:
# 새로운 feature 추가
df[['가장 가까운 버스 정류장 거리', '가장 가까운 버스 정류장 index', '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명', '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '인근 버스 정류장 개수']] = np.nan

In [27]:
df

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,가장 가까운 버스 정류장 index,가장 가까운 버스 정류장 노드 ID,가장 가까운 버스 정류소번호,가장 가까운 버스 정류소명,가장 가까운 버스 정류소 타입,가장 가까운 버스 정류장 X좌표,가장 가까운 버스 정류장 Y좌표,인근 버스 정류장 개수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# 원본 데이터에 적용
# '가장 가까운 버스 정류장 거리', '가장 가까운 버스 정류장 index'
# '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호'
# '가장 가까운 버스 정류소명', '가장 가까운 버스 정류소 타입', 
# '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '인근 버스 정류장 개수'

df['가장 가까운 버스 정류장 거리'] = df['가장 가까운 버스 정류장 거리'].fillna(df['좌표X,좌표Y'].map(shortest_dist_mapping))
df['가장 가까운 버스 정류장 index'] = df['가장 가까운 버스 정류장 index'].fillna(df['좌표X,좌표Y'].map(idx_mapping))
df['가장 가까운 버스 정류장 노드 ID'] = df['가장 가까운 버스 정류장 노드 ID'].fillna(df['좌표X,좌표Y'].map(ID_mapping))
df['가장 가까운 버스 정류소번호'] = df['가장 가까운 버스 정류소번호'].fillna(df['좌표X,좌표Y'].map(station_num_mapping))
df['가장 가까운 버스 정류소명'] = df['가장 가까운 버스 정류소명'].fillna(df['좌표X,좌표Y'].map(station_name_mapping))
df['가장 가까운 버스 정류소 타입'] = df['가장 가까운 버스 정류소 타입'].fillna(df['좌표X,좌표Y'].map(station_type_mapping))
df['가장 가까운 버스 정류장 X좌표'] = df['가장 가까운 버스 정류장 X좌표'].fillna(df['좌표X,좌표Y'].map(X_mapping))
df['가장 가까운 버스 정류장 Y좌표'] = df['가장 가까운 버스 정류장 Y좌표'].fillna(df['좌표X,좌표Y'].map(Y_mapping))
df['인근 버스 정류장 개수'] = df['인근 버스 정류장 개수'].fillna(df['좌표X,좌표Y'].map(nums_mapping))

In [29]:
df

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,가장 가까운 버스 정류장 index,가장 가까운 버스 정류장 노드 ID,가장 가까운 버스 정류소번호,가장 가까운 버스 정류소명,가장 가까운 버스 정류소 타입,가장 가까운 버스 정류장 X좌표,가장 가까운 버스 정류장 Y좌표,인근 버스 정류장 개수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,2.0,61.89584,11345.0,122000333.0,23445.0,개포우성아파트,일반차로,127.057864,37.476564,7.0
1,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,22,4,1987,...,2.0,61.89584,11345.0,122000333.0,23445.0,개포우성아파트,일반차로,127.057864,37.476564,7.0
2,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,54.98,201712,28,5,1987,...,2.0,61.89584,11345.0,122000333.0,23445.0,개포우성아파트,일반차로,127.057864,37.476564,7.0
3,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,3,4,1987,...,2.0,61.89584,11345.0,122000333.0,23445.0,개포우성아파트,일반차로,127.057864,37.476564,7.0
4,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201801,8,2,1987,...,2.0,61.89584,11345.0,122000333.0,23445.0,개포우성아파트,일반차로,127.057864,37.476564,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.65,202307,19,13,2014,...,4.0,59.47567,2633.0,106000471.0,7464.0,우디안아파트114동앞,일반차로,127.107073,37.619326,7.0
1128090,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.62,202307,25,12,2014,...,4.0,59.47567,2633.0,106000471.0,7464.0,우디안아파트114동앞,일반차로,127.107073,37.619326,7.0
1128091,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.65,202308,27,12,2014,...,4.0,59.47567,2633.0,106000471.0,7464.0,우디안아파트114동앞,일반차로,127.107073,37.619326,7.0
1128092,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.94,202309,2,18,2014,...,4.0,59.47567,2633.0,106000471.0,7464.0,우디안아파트114동앞,일반차로,127.107073,37.619326,7.0


In [30]:
df.to_csv('../data/preprocessed/3_bus_preprocessed.csv', index=False)

In [34]:
df[['아파트명', '좌표X', '좌표Y', '가장 가까운 버스 정류소명', '인근 버스 정류장 개수']].iloc[20000:20030]

,아파트명,좌표X,좌표Y,가장 가까운 버스 정류소명,인근 버스 정류장 개수
20000,강남,126.903908,37.484229,구로디지털단지역,13.0
20001,강남,126.903908,37.484229,구로디지털단지역,13.0
20002,강남,126.903908,37.484229,구로디지털단지역,13.0
20003,강남,126.903908,37.484229,구로디지털단지역,13.0
20004,강남,126.903908,37.484229,구로디지털단지역,13.0
20005,강남,126.903908,37.484229,구로디지털단지역,13.0
20006,강남,126.903908,37.484229,구로디지털단지역,13.0
20007,강남,126.903908,37.484229,구로디지털단지역,13.0
20008,강남,126.903908,37.484229,구로디지털단지역,13.0
20009,강남,126.903908,37.484229,구로디지털단지역,13.0


In [33]:
tmp = pd.read_csv('../data/train_test_data.csv')
tmp.iloc[20000:20030]

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_60948/1437299656.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../data/train_test_data.csv')


,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test
10000,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.95,201806,6,4,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54000.0,0
10001,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,59.99,201806,6,7,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46100.0,0
10002,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,59.54,201806,6,15,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46700.0,0
10003,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.99,201806,8,13,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54000.0,0
10004,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.97,201806,9,15,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52900.0,0
10005,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.92,201806,9,1,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49500.0,0
10006,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.97,201806,12,7,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54800.0,0
10007,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.99,201806,13,5,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53300.0,0
10008,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,59.82,201806,15,12,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48000.0,0
10009,서울특별시 강북구 미아동,811,811.0,0.0,두산위브트레지움,84.97,201806,15,16,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54500.0,0
